In [ ]:
import tkinter as tk
import math

def update_triangle(angle_degrees):
    canvas.delete("all")
    
    angle = math.radians(float(angle_degrees))
    hypotenuse = 300  # Panjang sisi miring dalam pixel
    
    # Hitung sisi segitiga
    adjacent = hypotenuse * math.cos(angle)  # Sisi alas
    opposite = hypotenuse * math.sin(angle)  # Sisi tegak
    
    # ======== GAMBAR SEGITIGA ========
    # Alas (biru)
    canvas.create_line(100, 350, 100 + adjacent, 350, fill="blue", width=3)
    # Tinggi (hijau)
    canvas.create_line(100 + adjacent, 350, 100 + adjacent, 350 - opposite, fill="green", width=3)
    # Miring (merah)
    canvas.create_line(100, 350, 100 + adjacent, 350 - opposite, fill="red", width=3)
    
    # Gambar sudut
    canvas.create_arc(80, 330, 120, 370, start=0, extent=angle_degrees, 
                     outline="orange", width=2, style="arc")
    
    # ======== TEOREMA PYTHAGORAS ========
    pythagoras_text = (
        "TEOREMA PYTHAGORAS:\n"
        "Miring² = Alas² + Tinggi²\n"
        f"{hypotenuse}² = {adjacent:.1f}² + {opposite:.1f}²\n"
        f"{hypotenuse**2} = {adjacent**2:.1f} + {opposite**2:.1f}\n"
        f"{hypotenuse**2} ≈ {adjacent**2 + opposite**2:.1f}"
    )
    
    # ======== RUMUS TRIGONOMETRI ========
    trigono_text = (
        "\nRUMUS TRIGONOMETRI:\n"
        f"sin({angle_degrees}°) = Tinggi/Miring = {opposite:.1f}/{hypotenuse} = {math.sin(angle):.3f}\n"
        f"cos({angle_degrees}°) = Alas/Miring = {adjacent:.1f}/{hypotenuse} = {math.cos(angle):.3f}\n"
        f"tan({angle_degrees}°) = Tinggi/Alas = {opposite:.1f}/{adjacent:.1f} = {math.tan(angle):.3f}"
    )
    
    # ======== CONTOH KASUS NYATA ========
    examples_text = (
        "\nCONTOH APLIKASI:\n"
        "1. Tinggi dinding yang bisa di jangkau tangga (hijau):\n"
        f"   diketahui : panjang tangga (merah) dan sudut {angle_degrees}°\n"
        f"   Tinggi dinding = Miring(panjang tangga) × sinθ\n"
        f"   {opposite:.1f} = {hypotenuse} × {math.sin(angle):.3f}\n\n"
        "2. Menentukan panjang tangga(merah):\n"
        f"   diketahui : jarak alas tangga ke dinding dan sudut {angle_degrees}°\n"
        f"   Miring(panjang tangga) = Alas(jarak dinding) / cosθ\n"
        f"   {hypotenuse:.1f} = {adjacent:.1f} / {math.cos(angle):.3f}\n\n"
        "3. Mengukur tinggi pohon:\n"
        f"   diketahui : jarak ke pohon dan sudut {angle_degrees}°\n"
        f"   Tinggi = Alas(jarak ke pohon) × tanθ\n"
        f"   {opposite:.1f} = {adjacent:.1f} × {math.tan(angle):.3f}"
    )
    
    # Gabungkan semua teks
    info_text = pythagoras_text + trigono_text + examples_text
    
    # Tampilkan teks
    canvas.create_text(400, 50, text=info_text, font=("Arial", 11), 
                      anchor="nw", justify=tk.LEFT, fill="black")
    
    # Label sisi-sisi segitiga
    canvas.create_text(100, 320, text=f"Sudut: {angle_degrees}°", 
                      font=("Arial", 10, "bold"), fill="black")
    canvas.create_text(100 + adjacent/2, 370, text=f"Alas: {adjacent:.1f} px", 
                      font=("Arial", 10, "bold"), fill="black")
    canvas.create_text(100 + adjacent + 20, 350 - opposite/2, 
                      text=f"Tinggi: {opposite:.1f} px", font=("Arial", 10, "bold"), fill="black")
    canvas.create_text(100 + adjacent/2 - 30, 350 - opposite/2 - 20, 
                      text=f"Miring: {hypotenuse} px", font=("Arial", 10, "bold"), fill="black")

    # Lingkaran (untuk visualisasi sudut)
    center_x, center_y = 100, 350
    radius = 300
    canvas.create_oval(center_x - radius, center_y - radius, 
                    center_x + radius, center_y + radius, outline="blue")

# Membuat window
root = tk.Tk()
root.title("Pythagoras & Trigonometri dalam Kehidupan Sehari-hari")
canvas = tk.Canvas(root, width=750, height=550, bg="white")
canvas.pack()


# Slider untuk sudut
angle_slider = tk.Scale(root, from_=1, to=89, resolution=1, orient=tk.HORIZONTAL,
                       label="Atur sudut θ (derajat):",
                       command=update_triangle)
angle_slider.set(30)
angle_slider.pack(fill=tk.X, padx=20, pady=10)

update_triangle(30)
root.mainloop()

In [ ]:
# 1. Simulasi Rotasi Sederhana - Angular Velocity Konstan
import tkinter as tk
import math

# Setup jendela
root = tk.Tk()
root.title("🔄 Simulasi Rotasi Sederhana - Angular Velocity Konstan")

canvas = tk.Canvas(root, width=600, height=400, bg="white")
canvas.pack()

# Pusat rotasi
cx, cy = 300, 200
radius = 100
angle_deg = 0
angular_velocity = 3  # derajat per frame

# Objek visual
circle = canvas.create_oval(cx - 5, cy - 5, cx + 5, cy + 5, fill="black")  # Titik pusat
dot = canvas.create_oval(0, 0, 0, 0, fill="red")  # Titik yang berputar
line = canvas.create_line(0, 0, 0, 0, fill="blue", width=2)  # Garis dari pusat ke titik
text = canvas.create_text(10, 10, anchor="nw", text="", font=("Arial", 12), fill="black")

def update():
    global angle_deg

    # Hitung posisi titik berdasarkan sudut
    angle_rad = math.radians(angle_deg)
    x = cx + radius * math.cos(angle_rad)
    y = cy + radius * math.sin(angle_rad)

    # Gambar ulang titik & garis
    canvas.coords(dot, x - 10, y - 10, x + 10, y + 10)
    canvas.coords(line, cx, cy, x, y)

    # Tampilkan informasi
    info = f"""
🌀 ROTASI DENGAN KECEPATAN SUDUT KONSTAN

Sudut: {angle_deg:.1f}°
Sudut (radian): {angle_rad:.2f}
Kecepatan sudut (ω): {angular_velocity}°/frame

x = cx + r × cos(θ)
  = {cx} + {radius} × cos({angle_deg:.1f}°) = {x:.1f}
y = cy + r × sin(θ)
  = {cy} + {radius} × sin({angle_deg:.1f}°) = {y:.1f}
"""
    canvas.itemconfig(text, text=info)

    # Tambah sudut
    angle_deg = (angle_deg + angular_velocity) % 360

    root.after(50, update)

update()
root.mainloop()


In [ ]:
# 2. ROTASI DENGAN PERCEPATAN SUDUT
import tkinter as tk
import math

# Vector class sederhana dengan metode rotate
class Vector:
    def __init__(self, x, y):
        self.x = x
        self.y = y

    def rotated(self, angle_rad):
        cos_a = math.cos(angle_rad)
        sin_a = math.sin(angle_rad)
        return Vector(
            self.x * cos_a - self.y * sin_a,
            self.x * sin_a + self.y * cos_a
        )

# Setup Tkinter
root = tk.Tk()
root.title("🔁 Rotasi dengan Percepatan Sudut")

canvas = tk.Canvas(root, width=600, height=400, bg="white")
canvas.pack()

# Variabel rotasi
center = Vector(300, 200)
radius = 100
angle_rad = 0
angular_velocity = 0.03  # rad/frame
angular_acceleration = 0.0001  # rad/frame²

# Gambar dasar
circle = canvas.create_oval(center.x - 5, center.y - 5, center.x + 5, center.y + 5, fill="black")
dot = canvas.create_oval(0, 0, 0, 0, fill="red")
line = canvas.create_line(0, 0, 0, 0, fill="blue", width=2)
text = canvas.create_text(10, 10, anchor="nw", text="", font=("Arial", 12), fill="black")

def update():
    global angle_rad, angular_velocity

    # Arah awal (1, 0) diputar sesuai sudut
    direction = Vector(1, 0).rotated(angle_rad)
    x = center.x + radius * direction.x
    y = center.y + radius * direction.y

    # Gambar ulang
    canvas.coords(dot, x - 10, y - 10, x + 10, y + 10)
    canvas.coords(line, center.x, center.y, x, y)

    # Tampilkan info
    info = f"""
🔁 ROTASI DENGAN PERCEPATAN SUDUT

Sudut (radian): {angle_rad:.4f}
Sudut (derajat): {math.degrees(angle_rad):.1f}°
Angular Velocity (ω): {angular_velocity:.5f} rad/frame
Angular Acceleration (α): {angular_acceleration:.5f} rad/frame²

x = cx + r × cos(θ)
  = {center.x} + {radius} × {direction.x:.3f} = {x:.1f}
y = cy + r × sin(θ)
  = {center.y} + {radius} × {direction.y:.3f} = {y:.1f}
"""
    canvas.itemconfig(text, text=info)

    # Update sudut & kecepatan sudut
    angular_velocity += angular_acceleration
    angle_rad = (angle_rad + angular_velocity) % (2 * math.pi)

    root.after(20, update)

update()
root.mainloop()


In [ ]:
# 2. ROTASI DENGAN PERCEPATAN SUDUT
import tkinter as tk
import math

# Vector class sederhana dengan metode rotate
class Vector:
    def __init__(self, x, y):
        self.x = x
        self.y = y

    def rotated(self, angle_rad):
        cos_a = math.cos(angle_rad)
        sin_a = math.sin(angle_rad)
        return Vector(
            self.x * cos_a - self.y * sin_a,
            self.x * sin_a + self.y * cos_a
        )

# Setup Tkinter
root = tk.Tk()
root.title("🔁 Rotasi dengan Percepatan Sudut")

canvas = tk.Canvas(root, width=600, height=400, bg="white")
canvas.pack()

# Variabel rotasi
center = Vector(300, 200)
radius = 100
angle_rad = 0
angular_velocity = 0.03  # rad/frame
angular_acceleration = 0.0001  # rad/frame²

# Gambar dasar
circle = canvas.create_oval(center.x - 5, center.y - 5, center.x + 5, center.y + 5, fill="black")
dot = canvas.create_oval(0, 0, 0, 0, fill="red")
line = canvas.create_line(0, 0, 0, 0, fill="blue", width=2)
text = canvas.create_text(10, 10, anchor="nw", text="", font=("Arial", 12), fill="black")

def update():
    global angle_rad, angular_velocity

    # Arah awal (1, 0) diputar sesuai sudut
    direction = Vector(1, 0).rotated(angle_rad)
    x = center.x + radius * direction.x
    y = center.y + radius * direction.y

    # Gambar ulang
    canvas.coords(dot, x - 10, y - 10, x + 10, y + 10)
    canvas.coords(line, center.x, center.y, x, y)

    # Tampilkan info
    info = f"""
🔁 ROTASI DENGAN PERCEPATAN SUDUT

Sudut (radian): {angle_rad:.4f}
Sudut (derajat): {math.degrees(angle_rad):.1f}°
Angular Velocity (ω): {angular_velocity:.5f} rad/frame
Angular Acceleration (α): {angular_acceleration:.5f} rad/frame²

x = cx + r × cos(θ)
  = {center.x} + {radius} × {direction.x:.3f} = {x:.1f}
y = cy + r × sin(θ)
  = {center.y} + {radius} × {direction.y:.3f} = {y:.1f}
"""
    canvas.itemconfig(text, text=info)

    # Update sudut & kecepatan sudut
    angular_velocity += angular_acceleration
    angle_rad = (angle_rad + angular_velocity) % (2 * math.pi)

    root.after(20, update)

update()
root.mainloop()


In [ ]:
#SCRIpT 1 : Simulasi Rotasi Baton, angular velocity constan
import tkinter as tk
import math
from vector import Vector 

# Class Baton (tongkat)
class Baton:
    def __init__(self, canvas, center, length):
        self.canvas = canvas
        self.center = center                # Vector posisi tengah
        self.length = length                # Panjang tongkat
        self.angle_deg = 0                  # Sudut awal (dalam derajat)
        self.angular_velocity = 2           # Kecepatan sudut (derajat/frame)
        self.angular_acceleration = 0       # Percepatan sudut (tetap nol)
        
        # Garis tongkat di canvas
        self.line = canvas.create_line(0, 0, 0, 0, width=8, fill="blue")
        
        # Teks informasi di sudut layar
        self.text = canvas.create_text(10, 10, anchor="nw", text="", font=("Arial", 12), fill="black")

    def update(self):
        # Konversi sudut ke radian
        angle_rad = math.radians(self.angle_deg)

        # Buat arah unit vector berdasarkan sudut
        direction = Vector(math.cos(angle_rad), math.sin(angle_rad))

        # Hitung dua ujung tongkat berdasarkan pusat
        end1 = self.center.added(direction.multed(self.length))
        end2 = self.center.subbed(direction.multed(self.length))

        # Perbarui garis tongkat pada canvas
        self.canvas.coords(self.line, end1.x, end1.y, end2.x, end2.y)

        # Tampilkan informasi perhitungan pada layar
        info = f"""Sudut (derajat): {self.angle_deg:.1f}°
Sudut (radian): {angle_rad:.2f}
Angular Velocity: {self.angular_velocity:.2f}°/frame
Angular Acceleration: {self.angular_acceleration:.2f}°/frame²
Ujung 1: ({end1.x:.1f}, {end1.y:.1f})
Ujung 2: ({end2.x:.1f}, {end2.y:.1f})
"""
        self.canvas.itemconfig(self.text, text=info)

        # Update sudut dengan angular velocity
        self.angle_deg = (self.angle_deg + self.angular_velocity) % 360


# Inisialisasi Tkinter dan kanvas
root = tk.Tk()
root.title("Simulasi Rotasi Baton - Nature of Code 3.1")

canvas = tk.Canvas(root, width=600, height=400, bg="white")
canvas.pack()

# Titik tengah canvas sebagai Vector
center = Vector(300, 200)

# Buat objek Baton
baton = Baton(canvas, center, length=100)

# Fungsi animasi
def animate():
    baton.update()
    root.after(50, animate)

animate()
root.mainloop()

In [ ]:
#SCRIPT 2 : Simulasi Rotasi Baton, angular_velocity = 0.05, angular_acceleration = 0.001 
import tkinter as tk
import math
from vector import Vector

# ==========================
# Class Baton (Tongkat)
# ==========================
class Baton:
    def __init__(self, center, length):
        self.center = center                # Titik pusat rotasi (Vector)
        self.length = length               # Panjang tongkat

        self.angle = 0                     # Sudut awal (radian)
        self.angular_velocity = 0.05       # Kecepatan sudut
        self.angular_acceleration = 0.0005 # Percepatan sudut

        self.line = None                   # ID garis pada canvas
        self.circle1 = None                # Lingkaran ujung kiri
        self.circle2 = None                # Lingkaran ujung kanan

    def update(self):
        # Update sudut dan kecepatan sudut
        self.angle += self.angular_velocity
        self.angular_velocity += self.angular_acceleration

    def display(self, canvas):
        # Hitung arah rotasi sebagai unit vector
        direction = Vector(math.cos(self.angle), math.sin(self.angle))
        half = direction.multed(self.length / 2)

        # Ujung kiri dan kanan dari pusat
        end1 = self.center.subbed(half)
        end2 = self.center.added(half)

        if self.line is None:
            # Pertama kali buat objek canvas
            self.line = canvas.create_line(end1.x, end1.y, end2.x, end2.y, width=4, fill="blue")
            self.circle1 = canvas.create_oval(end1.x-6, end1.y-6, end1.x+6, end1.y+6, fill="red")
            self.circle2 = canvas.create_oval(end2.x-6, end2.y-6, end2.x+6, end2.y+6, fill="red")
        else:
            # Update posisi objek
            canvas.coords(self.line, end1.x, end1.y, end2.x, end2.y)
            canvas.coords(self.circle1, end1.x-6, end1.y-6, end1.x+6, end1.y+6)
            canvas.coords(self.circle2, end2.x-6, end2.y-6, end2.x+6, end2.y+6)

        return self.angle, self.angular_velocity, self.angular_acceleration


# ==========================
# Fungsi Update Layar
# ==========================
def update():
    baton.update()
    angle, a_vel, a_acc = baton.display(canvas)

    # Tampilkan informasi di layar
    info_text = (
        f"Angle (rad) = {angle:.3f}\n"
        f"Angular Velocity = {a_vel:.3f}\n"
        f"Angular Acceleration = {a_acc:.5f}"
    )
    label_info.config(text=info_text)

    window.after(30, update)  # Loop animasi


# ==========================
# Setup Tkinter Window
# ==========================
WIDTH, HEIGHT = 600, 400
window = tk.Tk()
window.title("Simulasi Rotasi Sudut - The Nature of Code 3.2")

canvas = tk.Canvas(window, width=WIDTH, height=HEIGHT, bg="white")
canvas.pack()

label_info = tk.Label(window, text="", font=("Arial", 12), justify="left")
label_info.pack()

# Buat baton di tengah layar
center = Vector(WIDTH / 2, HEIGHT / 2)
baton = Baton(center, length=150)

# Mulai animasi
update()
window.mainloop()


In [12]:
# SCRIPT 3 : # Simulasi Gaya Tarik dengan Gerakan Angular
import tkinter as tk
import math
import random
from vector import Vector  

#1. Fungsi ini memastikan nilai tetap dalam batas tertentu. Contoh:
#constrain(10, 0, 5) akan mengembalikan 5
#constrain(-2, 0, 5) akan mengembalikan 0
def constrain(val, min_val, max_val):
    return max(min_val, min(val, max_val))
#2. Bola Merah (Attractor)
class Attractor:
    def __init__(self, canvas, x=320, y=120):
        self.canvas = canvas
        self.position = Vector(x, y)  # Posisi tengah layar
        self.mass = 20                # Massa besar
        self.G = 1                    # Kekuatan gravitasi
        r = self.mass * 2             # Ukuran bola
        # Gambar lingkaran merah
        self.id = canvas.create_oval(x-r, y-r, x+r, y+r, fill="red")
    # Attractor mengaplikasikan gaya tarik pada Mover
    def attract(self, mover):
        # Hitung jarak antara Attractor dan Mover
        force = Vector(self.position.x - mover.position.x, 
                    self.position.y - mover.position.y)
        distance = force.mag()  # Panjang garis antara keduanya
        
        # Batasi jarak minimal 5 dan maksimal 25
        distance = constrain(distance, 5, 25)
        
        force.normalize()  # Buat vektor panjangnya menjadi 1
        
        # Hitung kekuatan tarik (seperti rumus gravitasi)
        strength = (self.G * self.mass * mover.mass) / (distance * distance)
        force.mult(strength)  # Terapkan kekuatan ke vektor
        
        return force

    def display(self):
        r = self.mass * 2
        self.canvas.coords(self.id, self.position.x - r, self.position.y - r,
                                  self.position.x + r, self.position.y + r)
#3. Bola Kecil (Mover)
class Mover:
    def __init__(self, canvas, x, y, mass):
        self.canvas = canvas
        self.mass = mass
        self.radius = self.mass * 8  # Semakin besar massa, semakin besar ukuran
        self.position = Vector(x, y) # Posisi acak
        # Kecepatan awal acak antara -1 sampai 1
        self.velocity = Vector(random.uniform(-1, 1), random.uniform(-1, 1))
        self.acceleration = Vector(0, 0)

        self.angle = 0
        self.angle_velocity = 0
        self.angle_acceleration = 0

        # Gambar bola abu-abu dengan garis penunjuk arah
        r = self.radius
        self.id_circle = canvas.create_oval(x-r, y-r, x+r, y+r, fill="gray")
        self.id_line = canvas.create_line(x, y, x+r, y, width=2, fill="black")

    def apply_force(self, force):
        f = force.copy()
        f.div(self.mass)
        self.acceleration.add(f)
    #Pergerakan:
    def update(self):
        # Update posisi berdasarkan kecepatan
        self.velocity.add(self.acceleration)
        self.position.add(self.velocity)

        # Putar garis penunjuk berdasarkan percepatan
        self.angle_acceleration = self.acceleration.x / 10.0
        self.angle_velocity += self.angle_acceleration
        self.angle_velocity = constrain(self.angle_velocity, -0.1, 0.1)
        self.angle += self.angle_velocity

        self.acceleration.mult(0)  # Reset percepatan
    #Gambar Ulang Posisi:
    def show(self):
        x, y, r = self.position.x, self.position.y, self.radius
        # Update posisi lingkaran
        self.canvas.coords(self.id_circle, x-r, y-r, x+r, y+r)
        
        # Hitung ujung garis penunjuk
        end_x = x + r * math.cos(self.angle)
        end_y = y + r * math.sin(self.angle)
        # Update posisi garis
        self.canvas.coords(self.id_line, x, y, end_x, end_y)

# ========================
# Main Program
# ========================
root = tk.Tk()
root.title("Example 3.2: Forces with Angular Motion - Nature of Code")

canvas = tk.Canvas(root, width=640, height=240, bg="white")
canvas.pack()

attractor = Attractor(canvas)

# Buat 20 bola kecil dengan posisi dan massa acak
movers = []
for _ in range(20):
    x = random.uniform(0, 640)  # Posisi X acak
    y = random.uniform(0, 240)  # Posisi Y acak
    mass = random.uniform(0.1, 2)  # Massa acak
    movers.append(Mover(canvas, x, y, mass))

def animate():
    attractor.display()
    for mover in movers:
        force = attractor.attract(mover)  # Hitung gaya tarik
        mover.apply_force(force)         # Terapkan gaya
        mover.update()                   # Update posisi
        mover.show()                     # Gambar ulang
    root.after(30, animate)             # Ulangi setiap 30ms

animate()
root.mainloop()


In [1]:
import tkinter as tk
import math
from vector import Vector

# Tkinter setup
root = tk.Tk()
root.title("🎯 Arah Mover Menuju Target + Garis Bantu Segitiga")

canvas = tk.Canvas(root, width=600, height=400, bg="white")
canvas.pack()

# Bola1 (posisi awal) dan Bola2 (target)
pos1 = Vector(100, 100)
pos2 = Vector(400, 300)

# Objek visual
radius = 10
ball = canvas.create_oval(0, 0, 0, 0, fill="red")
target = canvas.create_oval(pos2.x - 15, pos2.y - 15, pos2.x + 15, pos2.y + 15, outline="green")
line = canvas.create_line(0, 0, 0, 0, fill="blue", width=2)  # garis arah

# Garis bantu segitiga
line_alas = canvas.create_line(0, 0, 0, 0, fill="gray", dash=(4, 2))
line_tinggi = canvas.create_line(0, 0, 0, 0, fill="gray", dash=(4, 2))

# Teks
text = canvas.create_text(10, 10, anchor="nw", font=("Arial", 11), fill="black", text="")

# Kecepatan
speed = 2

def update():
    global pos1

    # Hitung arah dari bola1 ke bola2
    direction = pos2.subbed(pos1)
    angle = math.atan2(direction.y, direction.x)  # Sudut arah (radian)
    direction_unit = direction.normalized()
    velocity = direction_unit.multed(speed)
    pos1 = Vector(pos1.x + velocity.x, pos1.y + velocity.y)

    # Update posisi bola & arah
    canvas.coords(ball, pos1.x - radius, pos1.y - radius, pos1.x + radius, pos1.y + radius)
    canvas.coords(line, pos1.x, pos1.y, pos1.x + direction_unit.x * 30, pos1.y + direction_unit.y * 30)

    # ===== Garis Bantu Segitiga =====
    # Alas: horizontal dari pos1 ke target.x (tetap di y = pos1.y)
    canvas.coords(line_alas, pos1.x, pos1.y, pos2.x, pos1.y)
    # Tinggi: vertical dari target.x, pos1.y ke pos2
    canvas.coords(line_tinggi, pos2.x, pos1.y, pos2.x, pos2.y)

    # Info trigonometri
    delta_x = pos2.x - pos1.x
    delta_y = pos2.y - pos1.y

    info = f"""
📘 Trigonometri & Vector
Target: ({pos2.x}, {pos2.y})
Posisi Bola1: ({pos1.x:.1f}, {pos1.y:.1f})

Δx (alas) = {pos2.x} - {pos1.x:.1f} = {delta_x:.1f}
Δy (tinggi) = {pos2.y} - {pos1.y:.1f} = {delta_y:.1f}

Sudut arah (radian) = atan2(Δy, Δx) = {angle:.2f}
Sudut arah (°)      = {math.degrees(angle):.1f}°

Arah unit vector:
= (cos(θ), sin(θ)) = ({math.cos(angle):.2f}, {math.sin(angle):.2f})
= {direction_unit}

Kecepatan: arah × {speed} = {velocity}
"""
    canvas.itemconfig(text, text=info)

    # Ulangi jika belum sampai
    if direction.magnitude() > speed:
        root.after(30, update)

update()
root.mainloop()


In [2]:
#4. Simulasi Mover Mengejar Target Bergerak
import tkinter as tk
import math
from vector import Vector

# Setup Tkinter
root = tk.Tk()
root.title("🎯 Simulasi Mover Mengejar Target Bergerak")

canvas = tk.Canvas(root, width=600, height=400, bg="white")
canvas.pack()

# Bola1 (mover) dan bola2 (target)
pos1 = Vector(100, 100)
waypoints = [Vector(250, 300), Vector(250, 100), Vector(500, 150), Vector(100, 350), Vector(400, 350)]
waypoint_index = 0
pos2 = waypoints[waypoint_index]

# Visual
radius = 10
ball = canvas.create_oval(0, 0, 0, 0, fill="red")
target = canvas.create_oval(pos2.x - 10, pos2.y - 10, pos2.x + 10, pos2.y + 10, fill="green")
line = canvas.create_line(0, 0, 0, 0, fill="blue", width=2)
text = canvas.create_text(10, 10, anchor="nw", font=("Arial", 11), fill="black", text="")

# Kecepatan bola1
speed = 2

def update():
    global pos1, pos2, waypoint_index

    # Hitung arah dari bola1 ke bola2
    direction = pos2.subbed(pos1)
    distance = direction.magnitude()
    angle = math.atan2(direction.y, direction.x)
    direction_unit = direction.normalized()
    velocity = direction_unit.multed(speed)
    
    # Update posisi bola1
    if distance > speed:
        pos1 = Vector(pos1.x + velocity.x, pos1.y + velocity.y)
    else:
        # Jika bola1 sudah dekat, pindah ke waypoint berikutnya
        waypoint_index = (waypoint_index + 1) % len(waypoints)
        pos2 = waypoints[waypoint_index]
        canvas.coords(target, pos2.x - 10, pos2.y - 10, pos2.x + 10, pos2.y + 10)

    # Update tampilan bola dan garis
    canvas.coords(ball, pos1.x - radius, pos1.y - radius, pos1.x + radius, pos1.y + radius)
    canvas.coords(line, pos1.x, pos1.y, pos1.x + direction_unit.x * 30, pos1.y + direction_unit.y * 30)

    # Informasi perhitungan
    info = f"""
📘 Trigonometri & Vector Menuju Target
Waypoint ke-{waypoint_index + 1}: ({pos2.x}, {pos2.y})
Posisi Bola1: ({pos1.x:.1f}, {pos1.y:.1f})

Δx = {pos2.x} - {pos1.x:.1f} = {pos2.x - pos1.x:.1f}
Δy = {pos2.y} - {pos1.y:.1f} = {pos2.y - pos1.y:.1f}

Sudut arah (radian) = atan2(Δy, Δx) = {angle:.2f}
Sudut arah (°)      = {math.degrees(angle):.1f}°

Arah unit vector:
= (cos(θ), sin(θ)) = ({math.cos(angle):.2f}, {math.sin(angle):.2f})
= {direction_unit}

Kecepatan: arah × {speed} = {velocity}
"""
    canvas.itemconfig(text, text=info)

    # Ulangi setiap 30ms
    root.after(30, update)

update()
root.mainloop()


In [ ]:
#SCRIPT 4 : # Simulasi Peluru Meriam dengan Rotasi
import tkinter as tk
import math
from vector import Vector  

class CannonBall:
    def __init__(self, canvas, x, y):
        self.canvas = canvas
        self.position = Vector(x, y)
        self.velocity = Vector(0, 0)
        self.acceleration = Vector(0, 0)
        self.r = 8
        self.topspeed = 10
        self.id = canvas.create_oval(x - self.r, y - self.r, x + self.r, y + self.r, fill="black")

    def apply_force(self, force):
        self.acceleration.add(force)

    def update(self):
        self.velocity.add(self.acceleration)
        self.velocity.limit(self.topspeed)
        self.position.add(self.velocity)
        self.acceleration.mult(0)

    def display(self):
        x, y, r = self.position.x, self.position.y, self.r
        self.canvas.coords(self.id, x - r, y - r, x + r, y + r)

class Cannon:
    def __init__(self, canvas, x, y):
        self.canvas = canvas
        self.position = Vector(x, y)
        self.angle = -math.pi / 4  # -45 derajat
        self.length = 50
        self.id = None

    def draw(self):
        x, y = self.position.x, self.position.y
        end_x = x + self.length * math.cos(self.angle)
        end_y = y + self.length * math.sin(self.angle)
        if self.id is None:
            self.id = self.canvas.create_line(x, y, end_x, end_y, width=10, fill="gray")
        else:
            self.canvas.coords(self.id, x, y, end_x, end_y)

    def rotate(self, delta_angle):
        self.angle += delta_angle
        self.angle = max(-math.pi / 2, min(self.angle, 0))  # constrain sudut

    def shoot_force(self, magnitude=10):
        force = Vector(math.cos(self.angle), math.sin(self.angle))
        force.mult(magnitude)
        return force

# Setup Tkinter
root = tk.Tk()
root.title("Exercise 3.2 Cannon - Nature of Code")

WIDTH, HEIGHT = 640, 360
canvas = tk.Canvas(root, width=WIDTH, height=HEIGHT, bg="white")
canvas.pack()

cannon = Cannon(canvas, 50, 300)
ball = CannonBall(canvas, cannon.position.x, cannon.position.y)
shot = False

def animate():
    global ball, shot
    canvas.delete("background_text")

    cannon.draw()

    if shot:
        gravity = Vector(0, 0.2)
        ball.apply_force(gravity)
        ball.update()

    ball.display()

    # Reset jika jatuh
    if ball.position.y > HEIGHT:
        ball = CannonBall(canvas, cannon.position.x, cannon.position.y)
        shot = False

    # Tampilkan informasi di layar
    angle_deg = math.degrees(cannon.angle)
    canvas.create_text(10, 10, anchor="nw", text=f"Angle: {angle_deg:.1f}°", tag="background_text", font=("Arial", 14))

    pos = ball.position
    vel = ball.velocity
    acc = ball.acceleration
    canvas.create_text(10, 30, anchor="nw", text=f"Pos: ({pos.x:.1f}, {pos.y:.1f})", tag="background_text", font=("Arial", 12))
    canvas.create_text(10, 50, anchor="nw", text=f"Vel: ({vel.x:.2f}, {vel.y:.2f})", tag="background_text", font=("Arial", 12))
    canvas.create_text(10, 70, anchor="nw", text=f"Acc: ({acc.x:.2f}, {acc.y:.2f})", tag="background_text", font=("Arial", 12))

    root.after(30, animate)

def on_key_press(event):
    global shot
    if event.keysym == 'Left':
        cannon.rotate(-0.1)
    elif event.keysym == 'Right':
        cannon.rotate(0.1)
    elif event.keysym == 'space' and not shot:
        shot = True
        force = cannon.shoot_force(10)
        ball.apply_force(force)

root.bind("<KeyPress>", on_key_press)

animate()
root.mainloop()


In [ ]:
#SCRIPT 5 : # Simulasi mobil kotak yang bergerak mengikuti arah mouse
import tkinter as tk
import math
from vector import Vector

class Mover:
    def __init__(self, canvas, width, height):
        self.canvas = canvas  # tempat menggambar
        self.width = width    # lebar layar
        self.height = height  # tinggi layar
        self.position = Vector(width / 2, height / 2)  # posisi awal di tengah
        self.velocity = Vector(0, 0)  # kecepatan awal (diam)
        self.acceleration = Vector(0, 0)  # percepatan awal
        self.topspeed = 4  # kecepatan maksimum
        self.r = 16  # ukuran
        self.id = canvas.create_polygon(0, 0, 0, 0, 0, 0, fill="gray", outline="black")  # bentuk kotak

    def update(self, mouse_x, mouse_y):
        mouse = Vector(mouse_x, mouse_y)  # posisi mouse
        dir = mouse.subbed(self.position)    # arah menuju mouse
        dir.normalize()  # buat panjang vektor = 1
        dir.mult(0.5)    # perkecil pengaruhnya
        self.acceleration = dir  # set percepatan
        
        self.velocity.add(self.acceleration)  # tambah kecepatan
        self.velocity.limit(self.topspeed)    # batasi kecepatan maks
        self.position.add(self.velocity)      # update posisi

    def display(self):
        angle = self.velocity.heading()  # sudut berdasarkan arah gerak
        
        w, h = 30, 10  # ukuran kotak
        cx, cy = self.position.x, self.position.y  # posisi tengah
        
        cos_a = math.cos(angle)  # hitung komponen x
        sin_a = math.sin(angle)  # hitung komponen y
        
        points = []
        for dx, dy in [(-w/2, -h/2), (w/2, -h/2), (w/2, h/2), (-w/2, h/2)]:
            x = cx + dx * cos_a - dy * sin_a  # rotasi titik x
            y = cy + dx * sin_a + dy * cos_a  # rotasi titik y
            points.extend((x, y))
        
        self.canvas.coords(self.id, *points)  # update posisi kotak

    def check_edges(self):
        if self.position.x > self.width:  # jika keluar kanan
            self.position.x = 0           # muncul di kiri
        elif self.position.x < 0:         # jika keluar kiri
            self.position.x = self.width   # muncul di kanan
            
        if self.position.y > self.height:  # jika keluar bawah
            self.position.y = 0            # muncul di atas
        elif self.position.y < 0:          # jika keluar atas
            self.position.y = self.height  # muncul di bawah

# Setup Tkinter
root = tk.Tk()
root.title("Example 3.3: Pointing in the Direction of Motion")

WIDTH, HEIGHT = 640, 240
canvas = tk.Canvas(root, width=WIDTH, height=HEIGHT, bg="white")
canvas.pack()

mover = Mover(canvas, WIDTH, HEIGHT)

def animate():
    canvas.delete("text")
    mouse_x = canvas.winfo_pointerx() - canvas.winfo_rootx()  # posisi mouse x
    mouse_y = canvas.winfo_pointery() - canvas.winfo_rooty()  # posisi mouse y

    mover.update(mouse_x, mouse_y)  # update posisi kotak
    mover.check_edges()             # cek tepi layar
    mover.display()                 # gambar ulang
    
    # Tampilkan teks posisi dan kecepatan
    canvas.create_text(10, 10, anchor="nw",
                       text=f"Pos: ({mover.position.x:.1f}, {mover.position.y:.1f})",
                       font=("Arial", 12), tag="text")
    canvas.create_text(10, 30, anchor="nw",
                       text=f"Vel: ({mover.velocity.x:.2f}, {mover.velocity.y:.2f})",
                       font=("Arial", 12), tag="text")

    root.after(30, animate)  # ulangi setiap 30ms

animate()
root.mainloop()


In [5]:
# SCRIPT 6 : Simulasi Konversi Koordinat Polar ke Kartesian
import tkinter as tk
import math

# === Setup Tkinter ===
root = tk.Tk()
root.title("Example 3.4: Polar to Cartesian")

WIDTH, HEIGHT = 640, 240
canvas = tk.Canvas(root, width=WIDTH, height=HEIGHT, bg="white")
canvas.pack()

# === Inisialisasi variabel polar ===
r = HEIGHT * 0.45     # Jari-jari (radius)
theta = 0             # Sudut awal

# Objek lingkaran dan garis
circle_id = canvas.create_oval(0, 0, 0, 0, fill="gray", outline="black", width=2)
line_id = canvas.create_line(0, 0, 0, 0, fill="black", width=2)

def animate():
    global theta

    canvas.delete("text")
    canvas.configure(bg="white")

    # Konversi polar ke kartesian
    x = r * math.cos(theta)
    y = r * math.sin(theta)

    # Pusat koordinat di tengah canvas
    center_x = WIDTH / 2
    center_y = HEIGHT / 2

    # Koordinat akhir (x, y) ditranslasi ke posisi canvas
    px = center_x + x
    py = center_y + y

    # Update garis dari pusat ke titik
    canvas.coords(line_id, center_x, center_y, px, py)

    # Update lingkaran di posisi (px, py)
    radius = 24
    canvas.coords(circle_id, px - radius, py - radius, px + radius, py + radius)

    # Tampilkan data
    canvas.create_text(10, 10, anchor="nw", text=f"theta = {theta:.2f} rad", font=("Arial", 12), tag="text")
    canvas.create_text(10, 30, anchor="nw", text=f"x = {x:.1f}", font=("Arial", 12), tag="text")
    canvas.create_text(10, 50, anchor="nw", text=f"y = {y:.1f}", font=("Arial", 12), tag="text")

    # Update sudut theta untuk rotasi
    theta += 0.02

    root.after(30, animate)

animate()
root.mainloop()


In [ ]:
# Simulasi Polar ke Cartesian dengan Garis Bantu
import tkinter as tk
import math

WIDTH, HEIGHT = 600, 600
CENTER_X, CENTER_Y = WIDTH // 2, HEIGHT // 2

root = tk.Tk()
root.title("Simulasi Polar ke Cartesian dengan Garis Bantu")

canvas = tk.Canvas(root, width=WIDTH, height=HEIGHT, bg="white")
canvas.pack()

# ---------- Lingkaran utama ----------
circle_radius = 200
canvas.create_oval(CENTER_X - circle_radius, CENTER_Y - circle_radius,
                   CENTER_X + circle_radius, CENTER_Y + circle_radius,
                   outline="lightgray")

# ---------- Sliders ----------
theta_scale = tk.Scale(root, from_=0, to=360, label="Sudut θ (derajat)", orient="horizontal", length=300)
theta_scale.set(45)
theta_scale.pack()

radius_scale = tk.Scale(root, from_=0, to=200, label="Radius r", orient="horizontal", length=300)
radius_scale.set(100)
radius_scale.pack()

# ---------- Label hasil koordinat ----------
label_coords = tk.Label(root, text="", font=("Arial", 12))
label_coords.pack()

# ---------- Objek Canvas ----------
point = canvas.create_oval(0, 0, 0, 0, fill="red")
line_r = canvas.create_line(0, 0, 0, 0, fill="blue", width=2)         # garis radius r
line_x = canvas.create_line(0, 0, 0, 0, fill="gray", dash=(4, 2))     # proyeksi ke sumbu x
line_baseline = canvas.create_line(0, 0, 0, 0, fill="black", width=1) # sumbu x (θ = 0°)

# ---------- Label garis panjang ----------
text_r = canvas.create_text(0, 0, text="", fill="blue", font=("Arial", 10))
text_x = canvas.create_text(0, 0, text="", fill="black", font=("Arial", 10))
text_y = canvas.create_text(0, 0, text="", fill="black", font=("Arial", 10))

# ---------- Fungsi Update ----------
def update():
    theta_deg = theta_scale.get()
    r = radius_scale.get()
    theta_rad = math.radians(theta_deg)

    # Hitung posisi Cartesian
    x = r * math.cos(theta_rad)
    y = r * math.sin(theta_rad)
    screen_x = CENTER_X + x
    screen_y = CENTER_Y + y

    # Update titik
    r_dot = 6
    canvas.coords(point,
                  screen_x - r_dot, screen_y - r_dot,
                  screen_x + r_dot, screen_y + r_dot)

    # Garis radius (r)
    canvas.coords(line_r, CENTER_X, CENTER_Y, screen_x, screen_y)

    # Garis proyeksi Y (vertikal ke sumbu X)
    canvas.coords(line_x, screen_x, screen_y, screen_x, CENTER_Y)

    # Garis dasar sumbu X dari center ke sudut 0° (baseline)
    canvas.coords(line_baseline, CENTER_X, CENTER_Y, CENTER_X + r, CENTER_Y)

    # Update label
    label_coords.config(
        text=f"r = {r}, θ = {theta_deg}°\n→ x = {x:.2f}, y = {y:.2f}"
    )

    # Teks panjang garis-garis
    canvas.coords(text_r, (CENTER_X + screen_x) / 2, (CENTER_Y + screen_y) / 2)
    canvas.itemconfig(text_r, text=f"r = {r}")

    canvas.coords(text_y, screen_x + 10, (CENTER_Y + screen_y) / 2)
    canvas.itemconfig(text_y, text=f"y = {y:.2f}")

    canvas.coords(text_x, (CENTER_X + screen_x) / 2, CENTER_Y + 15)
    canvas.itemconfig(text_x, text=f"x = {x:.2f}")

    root.after(33, update)

update()
root.mainloop()


In [ ]:
# SCRIPT 7 : Simulasi jalur spiral menggunakan koordinat polar
import tkinter as tk
import math

# Setup Tkinter
root = tk.Tk()
root.title("Exercise 3.4: Spiral Path")

WIDTH, HEIGHT = 640, 240
canvas = tk.Canvas(root, width=WIDTH, height=HEIGHT, bg="white")
canvas.pack()

# === Inisialisasi polar ===
r = 0             # Radius mulai dari 0 (untuk spiral)
theta = 0         # Sudut awal
dot_radius = 5    # Ukuran titik spiral

# Simpan semua titik spiral
spiral_points = []

def animate():
    global theta, r

    # Konversi polar ke kartesian
    x = r * math.cos(theta)
    y = r * math.sin(theta)

    # Geser titik ke tengah canvas
    px = WIDTH / 2 + x
    py = HEIGHT / 2 + y

    # Simpan titik spiral
    spiral_points.append((px, py))

    # Gambar semua titik spiral
    for px, py in spiral_points:
        canvas.create_oval(px - dot_radius, py - dot_radius,
                           px + dot_radius, py + dot_radius,
                           fill="black", outline="")

    # Tambah sudut dan radius
    theta += 0.1   # makin besar = makin cepat berputar
    r += 0.3        # makin besar = spiral makin melebar

    # #Info teks
    # canvas.create_text(10, 10, anchor="nw",
    #                    text=f"theta: {theta:.2f} | radius: {r:.2f}",
    #                    font=("Arial", 10), fill="blue", tag="info")

    root.after(30, animate)

animate()
root.mainloop()


In [ ]:
# SCRIPT 8 : Simulasi kapal luar angkasa yang bergerak dan berputar
import tkinter as tk
import math
from vector import Vector

class Spaceship:
    def __init__(self, canvas, width, height):
        self.canvas = canvas
        self.width = width
        self.height = height

        self.position = Vector(width / 2, height / 2)
        self.velocity = Vector(0, 0)
        self.acceleration = Vector(0, 0)

        self.damping = 0.995
        self.topspeed = 6
        self.heading = 0
        self.r = 16
        self.thrusting = False

        self.id = canvas.create_polygon(0, 0, 0, 0, 0, 0, fill="gray", outline="black")

    def update(self):
        self.velocity.add(self.acceleration)
        self.velocity.mult(self.damping)
        self.velocity.limit(self.topspeed)
        self.position.add(self.velocity)
        self.acceleration = Vector(0, 0)

    def applyForce(self, force):
        self.acceleration.add(force)

    def turn(self, angle):
        self.heading += angle

    def thrust(self):
        angle = self.heading - math.pi / 2  # Kapal digambar tegak
        force = Vector.fromAngle(angle)
        force.mult(0.1)
        self.applyForce(force)
        self.thrusting = True

    def wrapEdges(self):
        buffer = self.r * 2
        if self.position.x > self.width + buffer:
            self.position.x = -buffer
        elif self.position.x < -buffer:
            self.position.x = self.width + buffer

        if self.position.y > self.height + buffer:
            self.position.y = -buffer
        elif self.position.y < -buffer:
            self.position.y = self.height + buffer

    def show(self):
        # Posisi dan rotasi segitiga
        angle = self.heading
        cx, cy = self.position.x, self.position.y

        cos_a = math.cos(angle)
        sin_a = math.sin(angle)

        # Titik segitiga (tubuh kapal)
        points = []
        for dx, dy in [(-self.r, self.r), (0, -self.r), (self.r, self.r)]:
            x = cx + dx * cos_a - dy * sin_a
            y = cy + dx * sin_a + dy * cos_a
            points.extend((x, y))

        self.canvas.coords(self.id, *points)

        # Warna berdasarkan thrust
        if self.thrusting:
            self.canvas.itemconfig(self.id, fill="red")
        else:
            self.canvas.itemconfig(self.id, fill="gray")
        self.thrusting = False

        # Garis arah dari ujung segitiga ke depan
        nose_x = cx + 0 * cos_a - (-self.r) * sin_a
        nose_y = cy + 0 * sin_a + (-self.r) * cos_a

        line_length = 20
        dir_x = math.cos(self.heading - math.pi / 2) * line_length
        dir_y = math.sin(self.heading - math.pi / 2) * line_length

        # Buat atau perbarui garis
        if not hasattr(self, 'direction_line'):
            self.direction_line = self.canvas.create_line(nose_x, nose_y,
                                                          nose_x + dir_x, nose_y + dir_y,
                                                          fill='blue', width=2)
        else:
            self.canvas.coords(self.direction_line,
                               nose_x, nose_y,
                               nose_x + dir_x, nose_y + dir_y)


# Setup Tkinter
root = tk.Tk()
root.title("Exercise 3.5: Asteroids Spaceship")

WIDTH, HEIGHT = 640, 240
canvas = tk.Canvas(root, width=WIDTH, height=HEIGHT, bg="white")
canvas.pack()

ship = Spaceship(canvas, WIDTH, HEIGHT)

# Event state
keys = {"left": False, "right": False, "z": False}

def key_press(event):
    if event.keysym == "Left":
        keys["left"] = True
    elif event.keysym == "Right":
        keys["right"] = True
    elif event.keysym.lower() == "z":
        keys["z"] = True

def key_release(event):
    if event.keysym == "Left":
        keys["left"] = False
    elif event.keysym == "Right":
        keys["right"] = False
    elif event.keysym.lower() == "z":
        keys["z"] = False

root.bind("<KeyPress>", key_press)
root.bind("<KeyRelease>", key_release)

def animate():
    canvas.delete("text")

    if keys["left"]:
        ship.turn(-0.03)
    if keys["right"]:
        ship.turn(0.03)
    if keys["z"]:
        ship.thrust()

    ship.update()
    ship.wrapEdges()
    ship.show()

    canvas.create_text(10, 10, anchor="nw",
                       text=f"Pos: ({ship.position.x:.1f}, {ship.position.y:.1f})",
                       font=("Arial", 12), tag="text")

    root.after(30, animate)

animate()
root.mainloop()


In [ ]:
# Simulasi Gerak Harmonik Vertikal + Grafik
import tkinter as tk
import math
import time

# Setup awal
WIDTH, HEIGHT = 600, 400
CENTER_X, CENTER_Y = WIDTH // 2, HEIGHT // 2

root = tk.Tk()
root.title("Simulasi Gerak Harmonik Vertikal + Grafik")

canvas = tk.Canvas(root, width=WIDTH, height=HEIGHT, bg="white")
canvas.pack()

# Sliders
amp_slider = tk.Scale(root, from_=0, to=150, label="Amplitudo (px)", orient="horizontal", length=300)
amp_slider.set(100)
amp_slider.pack()

freq_slider = tk.Scale(root, from_=0.1, to=2.0, resolution=0.1, label="Frekuensi (Hz)", orient="horizontal", length=300)
freq_slider.set(0.5)
freq_slider.pack()

# Label rumus dan hasil
info_label = tk.Label(root, text="", font=("Arial", 12))
info_label.pack()

# Objek animasi
radius = 10
dot = canvas.create_oval(0, 0, 0, 0, fill="red")
line = canvas.create_line(CENTER_X, CENTER_Y, CENTER_X, CENTER_Y, fill="blue", width=2)
text_y = canvas.create_text(CENTER_X + 40, CENTER_Y, text="", font=("Arial", 10), fill="black")

# Grafik waktu vs simpangan
GRAPH_WIDTH = 400
GRAPH_HEIGHT = 150
GRAPH_X = 100
GRAPH_Y = HEIGHT - GRAPH_HEIGHT - 20
graph_points = []

# Garis bantu grafik
canvas.create_line(GRAPH_X, GRAPH_Y + GRAPH_HEIGHT//2, 
                  GRAPH_X + GRAPH_WIDTH, GRAPH_Y + GRAPH_HEIGHT//2, 
                  fill="gray", dash=(2, 2))  # Garis nol
canvas.create_text(GRAPH_X - 30, GRAPH_Y + GRAPH_HEIGHT//2, 
                  text="0", anchor="e", font=("Arial", 8))
canvas.create_text(GRAPH_X - 30, GRAPH_Y, 
                  text=f"{amp_slider.get()}", anchor="e", font=("Arial", 8))
canvas.create_text(GRAPH_X - 30, GRAPH_Y + GRAPH_HEIGHT, 
                  text=f"-{amp_slider.get()}", anchor="e", font=("Arial", 8))

start_time = time.time()

def update():
    global graph_points
    
    now = time.time()
    t = now - start_time  # waktu dalam detik

    A = amp_slider.get()
    f = freq_slider.get()
    omega = 2 * math.pi * f
    y_offset = A * math.sin(omega * t)

    # Update posisi titik
    y = CENTER_Y + y_offset
    canvas.coords(dot, CENTER_X - radius, y - radius, CENTER_X + radius, y + radius)
    canvas.coords(line, CENTER_X, CENTER_Y, CENTER_X, y)
    canvas.coords(text_y, CENTER_X + 40, y)
    canvas.itemconfig(text_y, text=f"y(t) = {y_offset:.1f}px")

    # Update grafik
    graph_points.append((t, y_offset))
    if len(graph_points) > 100:  # Batasi jumlah titik yang disimpan
        graph_points.pop(0)
    
    canvas.delete("graph")  # Hapus grafik sebelumnya
    
    # Gambar grafik baru
    for i in range(1, len(graph_points)):
        x1 = GRAPH_X + (i-1) * (GRAPH_WIDTH / 100)
        y1 = GRAPH_Y + GRAPH_HEIGHT//2 - (graph_points[i-1][1] * (GRAPH_HEIGHT/2/A))
        x2 = GRAPH_X + i * (GRAPH_WIDTH / 100)
        y2 = GRAPH_Y + GRAPH_HEIGHT//2 - (graph_points[i][1] * (GRAPH_HEIGHT/2/A))
        canvas.create_line(x1, y1, x2, y2, fill="green", width=2, tag="graph")
    
    # Update label skala grafik
    canvas.itemconfig(canvas.find_withtag("amplitude_text"), text=f"{A}")
    canvas.itemconfig(canvas.find_withtag("negative_amplitude_text"), text=f"-{A}")

    # Tampilkan rumus dan nilai
    info_label.config(
        text=f"Rumus: y(t) = A × sin(2πft)\nA = {A} px, f = {f} Hz, t = {t:.2f} s, y = {y_offset:.2f} px\n"
             f"Frekuensi sudut (ω) = {omega:.2f} rad/s"
    )

    root.after(20, update)

update()
root.mainloop()

In [ ]:
# Simulasi Gerak Harmonik Horizontal (Sumbu X)
import tkinter as tk
import math
import time

# Ukuran canvas dan titik pusat
WIDTH, HEIGHT = 600, 400
CENTER_X, CENTER_Y = WIDTH // 2, HEIGHT // 2

root = tk.Tk()
root.title("Simulasi Gerak Harmonik Horizontal (Sumbu X)")

canvas = tk.Canvas(root, width=WIDTH, height=HEIGHT, bg="white")
canvas.pack()

# Sliders
amp_slider = tk.Scale(root, from_=0, to=200, label="Amplitudo (px)", orient="horizontal", length=300)
amp_slider.set(100)
amp_slider.pack()

freq_slider = tk.Scale(root, from_=0.1, to=2.0, resolution=0.1, label="Frekuensi (Hz)", orient="horizontal", length=300)
freq_slider.set(0.5)
freq_slider.pack()

# Label rumus dan hasil
info_label = tk.Label(root, text="", font=("Arial", 12))
info_label.pack()

# Objek animasi
radius = 10
dot = canvas.create_oval(0, 0, 0, 0, fill="red")
line = canvas.create_line(CENTER_X, CENTER_Y, CENTER_X, CENTER_Y, fill="blue", width=2)
text_x = canvas.create_text(CENTER_X, CENTER_Y - 30, text="", font=("Arial", 10), fill="black")

start_time = time.time()

# Fungsi update animasi
def update():
    now = time.time()
    t = now - start_time  # waktu dalam detik

    A = amp_slider.get()
    f = freq_slider.get()
    omega = 2 * math.pi * f
    x_offset = A * math.sin(omega * t)

    # Posisi titik
    x = CENTER_X + x_offset
    canvas.coords(dot, x - radius, CENTER_Y - radius, x + radius, CENTER_Y + radius)
    canvas.coords(line, CENTER_X, CENTER_Y, x, CENTER_Y)
    canvas.coords(text_x, x, CENTER_Y - 30)
    canvas.itemconfig(text_x, text=f"x(t) = {x_offset:.1f}px")

    # Update label informasi
    info_label.config(
        text=f"Rumus: x(t) = A × sin(2πft)\nA = {A}, f = {f} Hz, t = {t:.2f}s, x = {x_offset:.2f}px"
    )

    root.after(20, update)

update()
root.mainloop()


In [ ]:
# SCRIPT 9 : # Simulasi Gerakan Harmonik Sederhana (SHM) Berdasarkan Frame Count
import tkinter as tk
import math

class SHMFrameCountApp:
    def __init__(self, root):
        self.root = root
        self.root.title("SHM - Based on Frame Count")

        self.canvas_width = 640
        self.canvas_height = 240
        self.canvas = tk.Canvas(root, width=self.canvas_width, height=self.canvas_height, bg="white")
        self.canvas.pack()

        self.period = 120
        self.amplitude = 200
        self.frame_count = 0

        self.center_x = self.canvas_width // 2
        self.center_y = self.canvas_height // 2

        # Buat elemen awal (garis dan lingkaran)
        self.line = self.canvas.create_line(self.center_x, self.center_y, self.center_x, self.center_y, width=2)
        self.circle = self.canvas.create_oval(0, 0, 0, 0, fill="gray", outline="black", width=2)

        self.update()

    def update(self):
        # Hitung posisi berdasarkan rumus SHM dengan frameCount
        x = self.amplitude * math.sin((2 * math.pi * self.frame_count) / self.period)
        y = 0  # tetap di sumbu horizontal

        # Update garis dari pusat ke titik bola
        self.canvas.coords(
            self.line,
            self.center_x, self.center_y,
            self.center_x + x, self.center_y + y
        )

        # Update lingkaran (bola)
        r = 24
        self.canvas.coords(
            self.circle,
            self.center_x + x - r,
            self.center_y + y - r,
            self.center_x + x + r,
            self.center_y + y + r
        )

        self.frame_count += 1
        self.root.after(16, self.update)  # sekitar 60 FPS

if __name__ == "__main__":
    root = tk.Tk()
    app = SHMFrameCountApp(root)
    root.mainloop()


In [ ]:
# SCRIPT 10 : Simulasi Gerak Harmonik Sederhana (Simple Harmonic Motion)
import tkinter as tk
import math

class SimpleHarmonicMotionApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Simple Harmonic Motion - Tkinter")
        self.canvas_width = 640
        self.canvas_height = 240
        self.canvas = tk.Canvas(root, width=self.canvas_width, height=self.canvas_height, bg="white")
        self.canvas.pack()

        self.angle = 0
        self.angle_velocity = 0.05
        self.amplitude = 200

        self.center_x = self.canvas_width // 2
        self.center_y = self.canvas_height // 2

        # Garis dan lingkaran
        self.line = self.canvas.create_line(self.center_x, self.center_y, self.center_x, self.center_y, width=2)
        self.circle = self.canvas.create_oval(0, 0, 0, 0, fill="gray", outline="black", width=2)

        self.update()

    def update(self):
        # Hitung posisi berdasarkan gerak harmonik sederhana
        x = self.amplitude * math.sin(self.angle)
        y = 0  # tetap di sumbu horizontal

        # Update garis
        self.canvas.coords(
            self.line,
            self.center_x, self.center_y,
            self.center_x + x, self.center_y + y
        )

        # Update lingkaran
        r = 24
        self.canvas.coords(
            self.circle,
            self.center_x + x - r,
            self.center_y + y - r,
            self.center_x + x + r,
            self.center_y + y + r
        )

        # Update sudut
        self.angle += self.angle_velocity

        # Loop
        self.root.after(16, self.update)  # sekitar 60 FPS

if __name__ == "__main__":
    root = tk.Tk()
    app = SimpleHarmonicMotionApp(root)
    root.mainloop()


In [9]:
# Simulasi Spiral 2D dengan Jejak
import tkinter as tk
import math
import time

WIDTH, HEIGHT = 600, 600
CENTER_X, CENTER_Y = WIDTH // 2, HEIGHT // 2

root = tk.Tk()
root.title("Simulasi Spiral 2D dengan Jejak")

canvas = tk.Canvas(root, width=WIDTH, height=HEIGHT, bg="white")
canvas.pack()

# Sliders
amp_x_slider = tk.Scale(root, from_=0, to=200, label="Amplitudo X (Aₓ)", orient="horizontal", length=300)
amp_x_slider.set(150)
amp_x_slider.pack()

amp_y_slider = tk.Scale(root, from_=0, to=200, label="Amplitudo Y (Aᵧ)", orient="horizontal", length=300)
amp_y_slider.set(100)
amp_y_slider.pack()

omega_x_slider = tk.Scale(root, from_=0.1, to=5.0, resolution=0.1, label="ωₓ (rad/s)", orient="horizontal", length=300)
omega_x_slider.set(2.0)
omega_x_slider.pack()

omega_y_slider = tk.Scale(root, from_=0.1, to=5.0, resolution=0.1, label="ωᵧ (rad/s)", orient="horizontal", length=300)
omega_y_slider.set(3.0)
omega_y_slider.pack()

# Label informasi
info_label = tk.Label(root, text="", font=("Arial", 12))
info_label.pack()

# Objek utama
radius = 8
dot = canvas.create_oval(0, 0, 0, 0, fill="red")
line = canvas.create_line(CENTER_X, CENTER_Y, CENTER_X, CENTER_Y, fill="blue", width=2)
angle_text = canvas.create_text(CENTER_X + 20, CENTER_Y - 30, text="", font=("Arial", 10), fill="black")

# Jejak (trail)
trail_length = 200
trail_points = []

start_time = time.time()

def update():
    now = time.time()
    t = now - start_time

    A_x = amp_x_slider.get()
    A_y = amp_y_slider.get()
    ω_x = omega_x_slider.get()
    ω_y = omega_y_slider.get()

    θ_x = ω_x * t
    θ_y = ω_y * t

    x_offset = A_x * math.cos(θ_x)
    y_offset = A_y * math.sin(θ_y)

    x = CENTER_X + x_offset
    y = CENTER_Y + y_offset

    # Simpan jejak
    trail_points.append((x, y))
    if len(trail_points) > trail_length:
        trail_points.pop(0)

    canvas.delete("trail")  # Hapus jejak lama
    for i, (tx, ty) in enumerate(trail_points):
        alpha = int(255 * (i + 1) / trail_length)
        gray = hex(255 - alpha)[2:].zfill(2)
        color = f"#ff{gray}{gray}"  # Gradasi merah ke putih
        canvas.create_oval(tx - 3, ty - 3, tx + 3, ty + 3, fill=color, outline="", tags="trail")

    # Update titik dan garis
    canvas.coords(dot, x - radius, y - radius, x + radius, y + radius)
    canvas.coords(line, CENTER_X, CENTER_Y, x, y)
    canvas.coords(angle_text, x + 20, y)
    canvas.itemconfig(angle_text, text="(x, y)")

    # Label informasi
    info_label.config(
        text=f"x(t) = {A_x}·cos({θ_x:.2f}) = {x_offset:.1f}px\n"
             f"y(t) = {A_y}·sin({θ_y:.2f}) = {y_offset:.1f}px\n"
             f"θₓ = {θ_x:.2f} rad, θᵧ = {θ_y:.2f} rad\n"
             f"ωₓ = {ω_x}, ωᵧ = {ω_y}, t = {t:.2f}s"
    )

    root.after(20, update)

update()
root.mainloop()


In [8]:
# SCRIPT 10 : Simulasi Gerakan Osilator
import tkinter as tk
import math
import random
from vector import Vector

# === Konstanta Layar ===
WIDTH, HEIGHT = 640, 240

# === Kelas Oscillator ===
class Oscillator:
    def __init__(self, canvas):
        self.canvas = canvas
        self.angle = Vector(0, 0)
        self.angle_velocity = Vector(random.uniform(-0.05, 0.05), random.uniform(-0.05, 0.05))
        self.amplitude = Vector(random.uniform(20, WIDTH / 2), random.uniform(20, HEIGHT / 2))

        # Buat objek visual (garis dan lingkaran)
        self.line_id = canvas.create_line(0, 0, 0, 0, fill="black", width=2)
        self.circle_id = canvas.create_oval(0, 0, 0, 0, fill="gray", outline="black")

    def update(self):
        self.angle.add(self.angle_velocity)

    def display(self):
        # Konversi sudut sinusoidal ke posisi (x, y)
        x = math.sin(self.angle.x) * self.amplitude.x
        y = math.sin(self.angle.y) * self.amplitude.y

        center_x, center_y = WIDTH / 2, HEIGHT / 2
        px, py = center_x + x, center_y + y

        # Update garis dari pusat ke titik osilasi
        self.canvas.coords(self.line_id, center_x, center_y, px, py)

        # Update lingkaran
        radius = 16
        self.canvas.coords(
            self.circle_id,
            px - radius, py - radius,
            px + radius, py + radius
        )

# === Setup Tkinter ===
root = tk.Tk()
root.title("Example 3.7: Oscillator Objects")

canvas = tk.Canvas(root, width=WIDTH, height=HEIGHT, bg="white")
canvas.pack()

# === Buat banyak oscillator ===
oscillators = [Oscillator(canvas) for _ in range(10)]

# === Fungsi animasi utama ===
def animate():
    for osc in oscillators:
        osc.update()
        osc.display()
    root.after(30, animate)

animate()
root.mainloop()


In [10]:
# SCRIPT 10 : Simulasi gelombang statis menggunakan sinus
import tkinter as tk
import math

# === Konstanta Layar ===
WIDTH, HEIGHT = 640, 240
ANGLE_VELOCITY = 0.2
AMPLITUDE = 100
SPACING = 24
RADIUS = 24

# === Setup Tkinter ===
root = tk.Tk()
root.title("Example 3.8: Static Wave")

canvas = tk.Canvas(root, width=WIDTH, height=HEIGHT, bg="white")
canvas.pack()

# === Gambar lingkaran gelombang ===
angle = 0
for x in range(0, WIDTH + 1, SPACING):
    # 1) Hitung posisi y menggunakan sinus
    y = AMPLITUDE * math.sin(angle)
    center_y = y + HEIGHT / 2

    # 2) Gambar lingkaran di posisi (x, center_y)
    canvas.create_oval(
        x - RADIUS, center_y - RADIUS,
        x + RADIUS, center_y + RADIUS,
        fill="gray", outline="black", width=2
    )

    # 3) Tambahkan sudut
    angle += ANGLE_VELOCITY

root.mainloop()


In [3]:
import tkinter as tk
import math
from vector import Vector

# Konfigurasi jendela dan kanvas
WIDTH, HEIGHT = 640, 240
STEP = 24         # Jarak antar titik gelombang
CIRCLE_SIZE = 48  # Diameter lingkaran

class WaveSimulation:
    def __init__(self, root):
        self.canvas = tk.Canvas(root, width=WIDTH, height=HEIGHT, bg="white")
        self.canvas.pack()

        self.start_angle = 0.0       # Mirip variabel startAngle di Processing
        self.angle_velocity = 0.2    # Kecepatan perubahan sudut antar titik

        self.circles = []  # List untuk menyimpan ID lingkaran di canvas

        # Inisialisasi lingkaran pada posisi awal
        for x in range(0, WIDTH + 1, STEP):
            y = HEIGHT // 2  # Posisi y sementara
            circle_id = self.canvas.create_oval(
                x - CIRCLE_SIZE//2, y - CIRCLE_SIZE//2,
                x + CIRCLE_SIZE//2, y + CIRCLE_SIZE//2,
                fill="gray", outline="black"
            )
            self.circles.append(circle_id)

        # Mulai animasi
        self.animate()

    def animate(self):
        # Bersihkan background dengan cara mengganti warna semua oval
        self.canvas.delete("all")

        angle = self.start_angle
        self.start_angle += 0.02  # Seiring waktu sudut awal bertambah

        # Gambar ulang semua lingkaran mengikuti pola gelombang
        for i, x in enumerate(range(0, WIDTH + 1, STEP)):
            # Ubah nilai sin menjadi nilai y (0 - HEIGHT)
            sin_value = math.sin(angle)
            y = self.map_value(sin_value, -1, 1, 0, HEIGHT)

            # Gambar ulang lingkaran
            circle_id = self.canvas.create_oval(
                x - CIRCLE_SIZE//2, y - CIRCLE_SIZE//2,
                x + CIRCLE_SIZE//2, y + CIRCLE_SIZE//2,
                fill="gray", outline="black"
            )

            angle += self.angle_velocity

        # Ulangi fungsi animate setiap 30 milidetik (~33 FPS)
        self.canvas.after(30, self.animate)

    @staticmethod
    def map_value(value, start1, stop1, start2, stop2):
        # Fungsi seperti map() di Processing: ubah nilai dari satu rentang ke rentang lain
        return start2 + (stop2 - start2) * ((value - start1) / (stop1 - start1))


if __name__ == "__main__":
    root = tk.Tk()
    root.title("Gelombang Sine dengan Tkinter")
    sim = WaveSimulation(root)
    root.mainloop()


In [4]:
import tkinter as tk
from vector import Vector
from perlin_noise import PerlinNoise

# Konfigurasi jendela
WIDTH, HEIGHT = 640, 240
STEP = 24
CIRCLE_SIZE = 48

class WaveWithNoise:
    def __init__(self, root):
        self.canvas = tk.Canvas(root, width=WIDTH, height=HEIGHT, bg="white")
        self.canvas.pack()

        # Inisialisasi noise generator dan offset
        self.noise = PerlinNoise(octaves=1)
        self.x_offset_start = 0.0   # Nilai awal untuk offset
        self.offset_increment = 0.1 # Seberapa cepat noise bergerak

        self.circle_ids = []  # Menyimpan ID lingkaran

        for x in range(0, WIDTH + 1, STEP):
            y = HEIGHT // 2
            circle_id = self.canvas.create_oval(
                x - CIRCLE_SIZE//2, y - CIRCLE_SIZE//2,
                x + CIRCLE_SIZE//2, y + CIRCLE_SIZE//2,
                fill="lightblue", outline="black"
            )
            self.circle_ids.append(circle_id)

        self.animate()

    def animate(self):
        # Bersihkan canvas
        self.canvas.delete("all")

        x_offset = self.x_offset_start
        self.x_offset_start += 0.01  # Perubahan waktu global (seperti frameCount)

        for i, x in enumerate(range(0, WIDTH + 1, STEP)):
            # Ambil nilai noise untuk x_offset
            noise_value = self.noise(x_offset)

            # Mapping noise (-1 to 1) menjadi (0 to HEIGHT)
            amplitude = HEIGHT / 3  # tinggi gelombang
            center_y = HEIGHT / 2   # titik tengah vertikal
            y = self.map_value(noise_value, 0, 1, center_y + amplitude, center_y - amplitude)

            # Gambar lingkaran
            circle_id = self.canvas.create_oval(
                x - CIRCLE_SIZE//2, y - CIRCLE_SIZE//2,
                x + CIRCLE_SIZE//2, y + CIRCLE_SIZE//2,
                fill="lightblue", outline="black"
            )

            self.circle_ids[i] = circle_id
            x_offset += self.offset_increment  # Tambah offset untuk gelombang menyebar

        self.canvas.after(30, self.animate)

    @staticmethod
    def map_value(value, start1, stop1, start2, stop2):
        return start2 + (stop2 - start2) * ((value - start1) / (stop1 - start1))

if __name__ == "__main__":
    root = tk.Tk()
    root.title("Perlin Noise Wave - Tkinter")
    app = WaveWithNoise(root)
    root.mainloop()


In [5]:
from tkinter import *
import math

from vector import Vector

# --- Konstanta global ---
WIDTH, HEIGHT = 640, 240
CIRCLE_SIZE = 24

# --- Kelas Wave seperti di JS ---
class Wave:
    def __init__(self, canvas, x, y, w, amplitude, period):
        self.canvas = canvas
        self.xspacing = 8
        self.w = w
        self.origin = Vector(x, y)
        self.theta = 0.0
        self.amplitude = amplitude
        self.period = period
        self.dx = (math.tau / self.period) * self.xspacing
        self.yvalues = [0.0 for _ in range(int(self.w / self.xspacing))]
        self.circle_ids = [None for _ in range(len(self.yvalues))]

    def update(self):
        self.theta += 0.02
        x = self.theta
        for i in range(len(self.yvalues)):
            self.yvalues[i] = math.sin(x) * self.amplitude
            x += self.dx

    def show(self):
        for i in range(len(self.yvalues)):
            x_pos = self.origin.x + i * self.xspacing
            y_pos = self.origin.y + self.yvalues[i]
            if self.circle_ids[i] is None:
                self.circle_ids[i] = self.canvas.create_oval(
                    x_pos - CIRCLE_SIZE//2, y_pos - CIRCLE_SIZE//2,
                    x_pos + CIRCLE_SIZE//2, y_pos + CIRCLE_SIZE//2,
                    fill="lightblue", outline="black"
                )
            else:
                self.canvas.coords(
                    self.circle_ids[i],
                    x_pos - CIRCLE_SIZE//2, y_pos - CIRCLE_SIZE//2,
                    x_pos + CIRCLE_SIZE//2, y_pos + CIRCLE_SIZE//2
                )

# --- Aplikasi utama ---
class App:
    def __init__(self, root):
        self.canvas = Canvas(root, width=WIDTH, height=HEIGHT, bg="white")
        self.canvas.pack()

        self.wave0 = Wave(self.canvas, 50, 75, 100, 20, 600)
        self.wave1 = Wave(self.canvas, 300, 120, 300, 40, 180)

        self.animate()

    def animate(self):
        self.wave0.update()
        self.wave0.show()

        self.wave1.update()
        self.wave1.show()

        self.canvas.after(30, self.animate)

# --- Jalankan ---
if __name__ == "__main__":
    root = Tk()
    root.title("Exercise 3.11 - OOP Wave")
    app = App(root)
    root.mainloop()


In [6]:
import tkinter as tk
import math
from vector import Vector

# Kelas Pendulum
class Pendulum:
    def __init__(self, origin_x, origin_y, r, canvas):
        self.origin = Vector(origin_x, origin_y)
        self.r = r
        self.angle = math.pi / 4  # Sudut awal 45 derajat
        self.a_velocity = 0.0     # Kecepatan sudut
        self.a_acceleration = 0.0 # Percepatan sudut
        self.damping = 0.995      # Redaman
        self.ball_radius = 24
        self.dragging = False
        self.canvas = canvas

        # Posisi bob
        self.bob = Vector()
        self.update_bob()

        # Objek canvas
        self.line = canvas.create_line(0, 0, 0, 0, width=2)
        self.circle = canvas.create_oval(0, 0, 0, 0, fill='gray')
        self.text = canvas.create_text(10, 10, anchor='nw', font=("Arial", 12), fill="black")

    def update_bob(self):
        self.bob.set(self.r * math.sin(self.angle), self.r * math.cos(self.angle))
        self.bob.add(self.origin)

    def update(self):
        gravity = 0.4
        self.a_acceleration = (-gravity / self.r) * math.sin(self.angle)
        self.a_velocity += self.a_acceleration
        self.angle += self.a_velocity
        self.a_velocity *= self.damping
        self.update_bob()

    def show(self):
        self.canvas.coords(self.line, self.origin.x, self.origin.y, self.bob.x, self.bob.y)
        self.canvas.coords(self.circle,
                           self.bob.x - self.ball_radius, self.bob.y - self.ball_radius,
                           self.bob.x + self.ball_radius, self.bob.y + self.ball_radius)
        info = (
            f"Sudut: {math.degrees(self.angle):.2f}°\n"
            f"Percepatan: {self.a_acceleration:.4f} rad/s²\n"
            f"Kecepatan: {self.a_velocity:.4f} rad/s"
        )
        self.canvas.itemconfig(self.text, text=info)

# Tkinter setup
root = tk.Tk()
root.title("Simulasi Pendulum")

canvas = tk.Canvas(root, width=640, height=360, bg='white')
canvas.pack()

pendulum = Pendulum(320, 50, 175, canvas)

# Loop animasi
def update():
    pendulum.update()
    pendulum.show()
    root.after(16, update)

update()
root.mainloop()


In [7]:
import tkinter as tk
import math
from vector import Vector

# Kelas Pendulum
class Pendulum:
    def __init__(self, origin_x, origin_y, r, canvas):
        self.origin = Vector(origin_x, origin_y)
        self.r = r
        self.angle = math.pi / 4  # Sudut awal 45 derajat
        self.a_velocity = 0.0     # Kecepatan sudut
        self.a_acceleration = 0.0 # Percepatan sudut
        self.damping = 0.995      # Redaman
        self.ball_radius = 24
        self.dragging = False
        self.canvas = canvas

        # Posisi bob
        self.bob = Vector()
        self.update_bob()

        # Objek canvas
        self.line = canvas.create_line(0, 0, 0, 0, width=2)
        self.circle = canvas.create_oval(0, 0, 0, 0, fill='gray')
        self.text = canvas.create_text(10, 10, anchor='nw', font=("Arial", 12), fill="black")

    def update_bob(self):
        self.bob.set(self.r * math.sin(self.angle), self.r * math.cos(self.angle))
        self.bob.add(self.origin)

    def update(self):
        if not self.dragging:
            gravity = 0.4
            self.a_acceleration = (-gravity / self.r) * math.sin(self.angle)
            self.a_velocity += self.a_acceleration
            self.angle += self.a_velocity
            self.a_velocity *= self.damping
        self.update_bob()

    def show(self):
        self.canvas.coords(self.line, self.origin.x, self.origin.y, self.bob.x, self.bob.y)
        self.canvas.coords(self.circle,
                           self.bob.x - self.ball_radius, self.bob.y - self.ball_radius,
                           self.bob.x + self.ball_radius, self.bob.y + self.ball_radius)
        info = (
            f"Sudut: {math.degrees(self.angle):.2f}°\n"
            f"Percepatan: {self.a_acceleration:.4f} rad/s²\n"
            f"Kecepatan: {self.a_velocity:.4f} rad/s"
        )
        self.canvas.itemconfig(self.text, text=info)

    def clicked(self, mx, my):
        d = math.hypot(mx - self.bob.x, my - self.bob.y)
        if d < self.ball_radius:
            self.dragging = True

    def stop_dragging(self):
        self.a_velocity = 0
        self.dragging = False

    def drag(self, mx, my):
        if self.dragging:
            dx = mx - self.origin.x
            dy = my - self.origin.y
            self.angle = math.atan2(dx, dy)  # Sudut dari sumbu vertikal, dibalik supaya arah mouse benar

# Tkinter setup
root = tk.Tk()
root.title("Simulasi Pendulum")

canvas = tk.Canvas(root, width=640, height=360, bg='white')
canvas.pack()

pendulum = Pendulum(320, 50, 175, canvas)

# Loop animasi
def update():
    pendulum.update()
    pendulum.show()
    root.after(16, update)

# Mouse events
def on_mouse_press(event):
    pendulum.clicked(event.x, event.y)

def on_mouse_release(event):
    pendulum.stop_dragging()

def on_mouse_drag(event):
    pendulum.drag(event.x, event.y)

# Bind mouse
canvas.bind("<ButtonPress-1>", on_mouse_press)
canvas.bind("<B1-Motion>", on_mouse_drag)
canvas.bind("<ButtonRelease-1>", on_mouse_release)

update()
root.mainloop()


In [8]:
import tkinter as tk
import math

# Window dimensions
WIDTH = 640
HEIGHT = 480

# Pendulum parameters
r1, r2 = 100, 100     # Panjang tali
m1, m2 = 10, 10       # Massa bola
a1, a2 = math.pi / 2, math.pi / 2  # Sudut awal (radian)
a1_v, a2_v = 0.0, 0.0 # Kecepatan sudut awal
g = 1                # Gravitasi

# Center point
cx, cy = WIDTH / 2, 100

# Koordinat sebelumnya untuk jejak
px2, py2 = -1, -1
trail = []

# Fungsi utama update setiap frame
def update():
    global a1, a2, a1_v, a2_v, px2, py2

    # Rumus percepatan sudut a1_a (dari physics pendulum ganda)
    num1 = -g * (2 * m1 + m2) * math.sin(a1)
    num2 = -m2 * g * math.sin(a1 - 2 * a2)
    num3 = -2 * math.sin(a1 - a2) * m2
    num4 = a2_v * a2_v * r2 + a1_v * a1_v * r1 * math.cos(a1 - a2)
    den = r1 * (2 * m1 + m2 - m2 * math.cos(2 * a1 - 2 * a2))
    a1_a = (num1 + num2 + num3 * num4) / den

    num1 = 2 * math.sin(a1 - a2)
    num2 = a1_v * a1_v * r1 * (m1 + m2)
    num3 = g * (m1 + m2) * math.cos(a1)
    num4 = a2_v * a2_v * r2 * m2 * math.cos(a1 - a2)
    den = r2 * (2 * m1 + m2 - m2 * math.cos(2 * a1 - 2 * a2))
    a2_a = (num1 * (num2 + num3 + num4)) / den

    # Perbarui kecepatan dan sudut
    a1_v += a1_a
    a2_v += a2_a
    a1 += a1_v
    a2 += a2_v

    # Damping agar gerakan lebih stabil
    a1_v *= 0.999
    a2_v *= 0.999

    # Hitung posisi bola
    x1 = r1 * math.sin(a1)
    y1 = r1 * math.cos(a1)
    x2 = x1 + r2 * math.sin(a2)
    y2 = y1 + r2 * math.cos(a2)

    # Pindah koordinat ke layar
    x1 += cx
    y1 += cy
    x2 += cx
    y2 += cy

    # Tambah trail
    if px2 != -1:
        trail.append((px2, py2, x2, y2))
        if len(trail) > 1000:
            trail.pop(0)
    px2, py2 = x2, y2

    # Gambar ulang semua
    canvas.delete("all")

    # Gambar trail jejak
    for x0, y0, x1t, y1t in trail:
        canvas.create_line(x0, y0, x1t, y1t, fill="blue")

    # Gambar batang dan bola
    canvas.create_line(cx, cy, x1, y1, width=2)
    canvas.create_oval(x1 - m1, y1 - m1, x1 + m1, y1 + m1, fill="black")

    canvas.create_line(x1, y1, x2, y2, width=2)
    canvas.create_oval(x2 - m2, y2 - m2, x2 + m2, y2 + m2, fill="black")

    # Tampilkan informasi
    canvas.create_text(10, 10, anchor="nw", text=f"a1 = {a1:.2f} rad")
    canvas.create_text(10, 30, anchor="nw", text=f"a2 = {a2:.2f} rad")

    # Jadwalkan frame berikutnya
    canvas.after(16, update)

# Setup Tkinter window
root = tk.Tk()
root.title("Double Pendulum - Python Tkinter")

canvas = tk.Canvas(root, width=WIDTH, height=HEIGHT, bg="white")
canvas.pack()

update()
root.mainloop()


In [12]:
import tkinter as tk
from vector import Vector

# ==== Bob (bola) ====
class Bob:
    def __init__(self, x, y, canvas):
        self.position = Vector(x, y)
        self.velocity = Vector(0, 0)
        self.acceleration = Vector(0, 0)
        self.mass = 20
        self.radius = 20
        self.dragging = False
        self.canvas = canvas
        self.id = canvas.create_oval(x - self.radius, y - self.radius,
                                     x + self.radius, y + self.radius,
                                     fill="blue")

    def apply_force(self, force):
        # F = m * a → a = F / m
        f = force.dived(self.mass)
        self.acceleration = self.acceleration.added(f)

    def update(self):
        if not self.dragging:
            # v = v + a
            self.velocity = self.velocity.added(self.acceleration)
            # s = s + v
            self.position = self.position.added(self.velocity)
        self.acceleration = Vector(0, 0)
        self.canvas.coords(self.id,
            self.position.x - self.radius,
            self.position.y - self.radius,
            self.position.x + self.radius,
            self.position.y + self.radius)

    def handle_click(self, mx, my):
        d = Vector(mx, my).subbed(self.position).mag()
        if d < self.radius:
            self.dragging = True

    def stop_dragging(self):
        self.dragging = False

    def handle_drag(self, mx, my):
        if self.dragging:
            self.position = Vector(mx, my)
            self.velocity = Vector(0, 0)

# ==== Spring ====
class Spring:
    def __init__(self, x, y, rest_length, canvas):
        self.anchor = Vector(x, y)
        self.rest_length = rest_length
        self.k = 0.2  # konstanta pegas
        self.canvas = canvas
        self.line_id = canvas.create_line(x, y, x, y)
        self.anchor_id = canvas.create_oval(x - 5, y - 5, x + 5, y + 5, fill="black")

    def connect(self, bob):
        # hitung gaya pegas
        force = bob.position.subbed(self.anchor)  # arah gaya: dari anchor ke bob
        d = force.mag()  # panjang saat ini
        stretch = d - self.rest_length  # selisih panjang

        # Fspring = -k * x
        force = force.normalized().multed(-self.k * stretch)
        bob.apply_force(force)
        return force  # dikembalikan agar bisa ditampilkan

    def constrain_length(self, bob, minlen, maxlen):
        direction = bob.position.subbed(self.anchor)
        d = direction.mag()
        if d < minlen:
            direction = direction.normalized().multed(minlen)
            bob.position = self.anchor.added(direction)
            bob.velocity = Vector(0, 0)
        elif d > maxlen:
            direction = direction.normalized().multed(maxlen)
            bob.position = self.anchor.added(direction)
            bob.velocity = Vector(0, 0)

    def show_line(self, bob):
        self.canvas.coords(self.line_id,
                           bob.position.x, bob.position.y,
                           self.anchor.x, self.anchor.y)

# ==== Main App ====
def main():
    root = tk.Tk()
    root.title("Spring Simulation")
    canvas = tk.Canvas(root, width=640, height=300, bg="white")
    canvas.pack()

    spring = Spring(320, 10, 100, canvas)
    bob = Bob(320, 100, canvas)

    text_id_force = canvas.create_text(10, 260, anchor="w", text="", font=("Consolas", 12))
    text_id_velocity = canvas.create_text(10, 280, anchor="w", text="", font=("Consolas", 12))
    text_id_accel = canvas.create_text(300, 280, anchor="w", text="", font=("Consolas", 12))

    mouse = {"x": 0, "y": 0}

    def update():
        # gaya gravitasi konstan ke bawah
        gravity = Vector(0, 0.98)
        bob.apply_force(gravity)

        bob.update()
        bob.handle_drag(mouse["x"], mouse["y"])

        # hitung dan aplikasikan gaya pegas ke bob
        spring_force = spring.connect(bob)

        # batasi panjang pegas
        spring.constrain_length(bob, 30, 200)

        # update garis pegas
        spring.show_line(bob)

        # tampilkan gaya, kecepatan, percepatan
        canvas.itemconfig(text_id_force,
                          text=f"F_spring = ({spring_force.x:.2f}, {spring_force.y:.2f})")
        canvas.itemconfig(text_id_velocity,
                          text=f"velocity  = ({bob.velocity.x:.2f}, {bob.velocity.y:.2f})")
        canvas.itemconfig(text_id_accel,
                          text=f"accel     = ({bob.acceleration.x:.2f}, {bob.acceleration.y:.2f})")

        root.after(16, update)  # ~60 FPS

    def on_mouse_press(event):
        bob.handle_click(event.x, event.y)

    def on_mouse_release(event):
        bob.stop_dragging()

    def on_mouse_motion(event):
        mouse["x"] = event.x
        mouse["y"] = event.y

    canvas.bind("<ButtonPress-1>", on_mouse_press)
    canvas.bind("<ButtonRelease-1>", on_mouse_release)
    canvas.bind("<Motion>", on_mouse_motion)

    update()
    root.mainloop()

if __name__ == "__main__":
    main()


In [15]:
# spring_simulation.py
import tkinter as tk
from vector import Vector

class Bob:
    def __init__(self, canvas, x, y, mass=24):
        self.canvas = canvas
        self.position = Vector(x, y)
        self.velocity = Vector()
        self.acceleration = Vector()
        self.mass = mass
        self.damping = 0.98
        self.drag_offset = Vector()
        self.dragging = False
        self.radius = mass
        self.id = canvas.create_oval(x - mass, y - mass, x + mass, y + mass, fill="gray")

    def apply_force(self, force):
        # F = m * a => a = F / m
        f = force.dived(self.mass)
        self.acceleration = self.acceleration.added(f)

    def update(self):
        if not self.dragging:
            self.velocity = self.velocity.added(self.acceleration)
            self.velocity = self.velocity.multed(self.damping)
            self.position = self.position.added(self.velocity)
        self.acceleration = Vector()  # Reset percepatan
        self.canvas.coords(self.id,
            self.position.x - self.radius,
            self.position.y - self.radius,
            self.position.x + self.radius,
            self.position.y + self.radius)

    def handle_click(self, mx, my):
        dx = mx - self.position.x
        dy = my - self.position.y
        if dx * dx + dy * dy < self.radius * self.radius:
            self.dragging = True
            self.drag_offset = Vector(self.position.x - mx, self.position.y - my)

    def stop_dragging(self):
        self.dragging = False

    def handle_drag(self, mx, my):
        if self.dragging:
            self.position = Vector(mx, my).added(self.drag_offset)

class Spring:
    def __init__(self, canvas, a, b, length, k=0.2):
        self.canvas = canvas
        self.a = a
        self.b = b
        self.length = length
        self.k = k
        self.id = canvas.create_line(0, 0, 0, 0)

    def update(self):
        force = self.a.position.subbed(self.b.position)  # arah
        distance = force.mag()
        stretch = distance - self.length
        force = force.normalized().multed(-1 * self.k * stretch)  # Hooke’s Law: F = -k * x

        # Terapkan gaya pada masing-masing bob
        self.a.apply_force(force)
        self.b.apply_force(force.multed(-1))

        # Perbarui visual garis pegas
        self.canvas.coords(self.id,
            self.a.position.x, self.a.position.y,
            self.b.position.x, self.b.position.y)

def main():
    root = tk.Tk()
    root.title("Exercise 3.16 - Multiple Springs")
    canvas = tk.Canvas(root, width=640, height=360, bg="white")
    canvas.pack()

    b1 = Bob(canvas, 320, 100)
    b2 = Bob(canvas, 320, 200)
    b3 = Bob(canvas, 320, 300)

    s1 = Spring(canvas, b1, b2, 100)
    s2 = Spring(canvas, b2, b3, 100)
    s3 = Spring(canvas, b1, b3, 100)

    mouse = {"x": 0, "y": 0, "pressed": False}

    def on_mouse_press(event):
        mouse["pressed"] = True
        b1.handle_click(event.x, event.y)

    def on_mouse_release(event):
        mouse["pressed"] = False
        b1.stop_dragging()

    def on_mouse_motion(event):
        mouse["x"], mouse["y"] = event.x, event.y

    def update():
        canvas.delete("info")

        b1.handle_drag(mouse["x"], mouse["y"])
        b1.update()
        b2.update()
        b3.update()

        s1.update()
        s2.update()
        s3.update()

        # Tampilkan informasi gaya, kecepatan, dan percepatan b1
        info = f"F: ({b1.acceleration.x * b1.mass:.2f}, {b1.acceleration.y * b1.mass:.2f}) | "
        info += f"v: {b1.velocity} | a: {b1.acceleration}"
        canvas.create_text(10, 10, anchor="nw", text=info, fill="black", font=("Arial", 10), tags="info")

        root.after(16, update)

    canvas.bind("<ButtonPress-1>", on_mouse_press)
    canvas.bind("<ButtonRelease-1>", on_mouse_release)
    canvas.bind("<Motion>", on_mouse_motion)

    update()
    root.mainloop()

if __name__ == "__main__":
    main()


In [17]:
from tkinter import *
from vector import Vector
import math

class Bob:
    def __init__(self, x, y, canvas):
        self.position = Vector(x, y)
        self.velocity = Vector(0, 0)
        self.acceleration = Vector(0, 0)
        self.mass = 24
        self.damping = 0.98
        self.dragging = False
        self.drag_offset = Vector(0, 0)
        self.canvas = canvas
        self.id = canvas.create_oval(x - self.mass / 2, y - self.mass / 2,
                                     x + self.mass / 2, y + self.mass / 2,
                                     fill="gray")

    def apply_force(self, force):
        f = force.dived(self.mass)
        self.acceleration = self.acceleration.added(f)

    def update(self):
        if not self.dragging:
            self.velocity = self.velocity.added(self.acceleration)
            self.velocity = self.velocity.multed(self.damping)
            self.position = self.position.added(self.velocity)
            self.acceleration = Vector(0, 0)

        r = self.mass / 2
        self.canvas.coords(self.id,
                           self.position.x - r, self.position.y - r,
                           self.position.x + r, self.position.y + r)

    def handle_click(self, mx, my):
        d = math.dist([mx, my], [self.position.x, self.position.y])
        if d < self.mass:
            self.dragging = True
            self.drag_offset = Vector(self.position.x - mx, self.position.y - my)

    def stop_dragging(self):
        self.dragging = False

    def handle_drag(self, mx, my):
        if self.dragging:
            self.position = Vector(mx, my).added(self.drag_offset)

class Spring:
    def __init__(self, a, b, length, canvas):
        self.a = a
        self.b = b
        self.length = length
        self.k = 0.2
        self.canvas = canvas
        self.id = canvas.create_line(a.position.x, a.position.y,
                                     b.position.x, b.position.y,
                                     fill="black", width=2)

    def update(self):
        force = self.a.position.subbed(self.b.position)
        stretch = force.mag() - self.length
        force = force.normalized().multed(-1 * self.k * stretch)
        self.a.apply_force(force)
        self.b.apply_force(force.multed(-1))
        self.canvas.coords(self.id,
                           self.a.position.x, self.a.position.y,
                           self.b.position.x, self.b.position.y)

def main():
    root = Tk()
    root.title("Spring Array Simulation")
    canvas = Canvas(root, width=640, height=360, bg="white")
    canvas.pack()

    bobs = [Bob(320, i * 40 + 50, canvas) for i in range(5)]
    springs = [Spring(bobs[i], bobs[i+1], 40, canvas) for i in range(4)]
    mouse = {"x": 0, "y": 0}

    def on_mouse_press(event):
        for b in bobs:
            b.handle_click(event.x, event.y)

    def on_mouse_release(event):
        for b in bobs:
            b.stop_dragging()

    def on_mouse_motion(event):
        mouse["x"], mouse["y"] = event.x, event.y

    def update():
        for b in bobs:
            b.handle_drag(mouse["x"], mouse["y"])
        for s in springs:
            s.update()
        for b in bobs:
            b.update()
        canvas.after(16, update)

    canvas.bind("<ButtonPress-1>", on_mouse_press)
    canvas.bind("<ButtonRelease-1>", on_mouse_release)
    canvas.bind("<Motion>", on_mouse_motion)

    update()
    root.mainloop()

if __name__ == "__main__":
    main()


In [23]:
import tkinter as tk
import math
import random
from vector import Vector

class Oscillator:
    def __init__(self, canvas, amplitude):
        self.canvas = canvas
        self.theta = 0
        self.amplitude = amplitude
        self.line = canvas.create_line(0, 0, 0, 0, fill='black')
        self.dot = canvas.create_oval(0, 0, 0, 0, fill='black')

    def update(self, vel_mag):
        self.theta += vel_mag

    def display(self, pos):
        x = math.cos(self.theta) * self.amplitude
        self.canvas.coords(self.line, pos.x, pos.y, pos.x + x, pos.y)
        r = 4
        self.canvas.coords(self.dot, pos.x + x - r, pos.y - r, pos.x + x + r, pos.y + r)

class Crawler:
    def __init__(self, canvas, width, height):
        self.canvas = canvas
        self.pos = Vector(random.uniform(0, width), random.uniform(0, height))
        self.vel = Vector(random.uniform(-1, 1), random.uniform(-1, 1))
        self.acc = Vector()
        self.mass = random.uniform(8, 16)
        self.radius = self.mass
        self.id = canvas.create_oval(0, 0, 0, 0, fill='gray')
        self.osc = Oscillator(canvas, self.mass * 2)

    def apply_force(self, force):
        f = force.dived(self.mass)
        self.acc = self.acc.added(f)

    def update(self):
        self.vel = self.vel.added(self.acc)
        self.pos = self.pos.added(self.vel)
        self.acc = Vector()
        self.osc.update(self.vel.mag() / 10)

    def display(self):
        r = self.radius
        self.canvas.coords(self.id, self.pos.x - r, self.pos.y - r, self.pos.x + r, self.pos.y + r)
        self.osc.display(self.pos)

class Attractor:
    def __init__(self, canvas, pos, mass, g):
        self.canvas = canvas
        self.pos = pos
        self.mass = mass
        self.G = g
        self.radius = mass
        self.dragging = False
        self.rollover = False
        self.drag_offset = Vector()
        self.id = canvas.create_oval(0, 0, 0, 0, fill='lightblue')

    def attract(self, crawler):
        force = self.pos.subbed(crawler.pos)
        d = max(5.0, min(25.0, force.mag()))
        force = force.normalized()
        strength = (self.G * self.mass * crawler.mass) / (d * d)
        return force.multed(strength)

    def render(self):
        fill = 'gray' if self.dragging else 'lightgreen' if self.rollover else 'lightblue'
        self.canvas.itemconfig(self.id, fill=fill)
        r = self.radius
        self.canvas.coords(self.id, self.pos.x - r, self.pos.y - r, self.pos.x + r, self.pos.y + r)

    def clicked(self, mx, my):
        d = math.hypot(mx - self.pos.x, my - self.pos.y)
        if d < self.radius:
            self.dragging = True
            self.drag_offset = Vector(self.pos.x - mx, self.pos.y - my)

    def stop_dragging(self):
        self.dragging = False

    def drag(self, mx, my):
        if self.dragging:
            self.pos.x = mx + self.drag_offset.x
            self.pos.y = my + self.drag_offset.y

    def rollover_check(self, mx, my):
        d = math.hypot(mx - self.pos.x, my - self.pos.y)
        self.rollover = d < self.radius

    def go(self):
        self.render()

# --- Main Application ---
def main():
    root = tk.Tk()
    root.title("Attraction Array with Oscillation")
    width, height = 640, 360
    canvas = tk.Canvas(root, width=width, height=height, bg='white')
    canvas.pack()

    attractor = Attractor(canvas, Vector(width / 2, height / 2), 20, 0.4)
    crawlers = [Crawler(canvas, width, height) for _ in range(6)]

    mouse = {'x': 0, 'y': 0}

    def draw():
        attractor.rollover_check(mouse['x'], mouse['y'])
        attractor.drag(mouse['x'], mouse['y'])
        attractor.go()

        for c in crawlers:
            f = attractor.attract(c)
            c.apply_force(f)
            c.update()
            c.display()

        root.after(16, draw)

    def on_mouse_move(event):
        mouse['x'], mouse['y'] = event.x, event.y

    def on_mouse_down(event):
        attractor.clicked(event.x, event.y)

    def on_mouse_up(event):
        attractor.stop_dragging()

    canvas.bind("<Motion>", on_mouse_move)
    canvas.bind("<ButtonPress-1>", on_mouse_down)
    canvas.bind("<ButtonRelease-1>", on_mouse_up)

    draw()
    root.mainloop()

if __name__ == "__main__":
    main()
